In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
YEAR = 2019

In [ ]:
#Can import from link or set to file path of a CSV
pbp_data = pd.read_csv('https://raw.githubusercontent.com/' + \
                       'ryurko/nflscrapR-data/master/' + \
                       'play_by_play_data/regular_season/reg_pbp_' + str(YEAR) + '.csv',
                       low_memory=False)

In [ ]:
#Import week and season info
game_info = pd.read_csv('http://www.habitatring.com/games.csv')[['game_id','week','season']]

In [ ]:
#Filter out plays where EPA is null
#Only leave plays with type no_play,pass,run
pbp_data = pbp_data.loc[(pbp_data.play_type.isin(['no_play','pass','run'])) &
                        (pbp_data.epa.isna()==False)]

In [ ]:
#Insert columns to show dropback or rush play
#Default value 0
pbp_data.insert(0,'dropback',0)
pbp_data.insert(0,'rush',0)

#Change value to 1 if dropback or rush
#Space between pipe | and 'pass' is to avoid player with 'pass' in his last name
#throwing off detection
pbp_data.dropback.loc[pbp_data.desc.str.contains(' pass|sacked|scrambles')] = 1

pbp_data.rush.loc[(pbp_data.desc.str.contains('left end|left tackle|left guard|up the middle|' + \
                                              'right guard|right tackle|right end')) &
                  (pbp_data.dropback==0)] = 1

In [ ]:
#Insert success column
#Success defined as a play with EPA > 0
pbp_data.insert(0,'success',0)

pbp_data.success.loc[pbp_data.epa>0] = 1

In [ ]:
#Mark interceptions as also being incomplete passes
pbp_data.incomplete_pass.loc[pbp_data.interception==1] = 1

In [ ]:
#Mark yards gained as null for plays with penalties
pbp_data.yards_gained.loc[pbp_data.play_type=='no_play'] = np.nan

In [ ]:
#Reset index inplace
pbp_data.reset_index(drop=True,inplace=True)

In [ ]:
#Merge play by play with game info
#This will add week and season info
pbp_data = pbp_data.merge(game_info,left_on='game_id',right_on='game_id')

In [ ]:
#Fill in missing player date
#Make list of plays with missing passers
missing_passers = pbp_data.loc[(pbp_data.play_type=='no_play') & 
                               (pbp_data.dropback==1)].index

#Go through each play with missing passer name
for i in missing_passers:
    #Split string on spaces 
    desc = pbp_data.desc.loc[i].split()
    
    #Search each word of the description
    for j in range(0,len(desc)):
        #Attach suffix to abbreviated player name
        if desc[j] in ('Jr','Jr.','Sr','Sr.','II','III','IV'):
            desc[j] = desc[j-1] + ' ' + desc[j] 
            
        #Passer name comes before pass, scramble, or sack
        if desc[j] in ('pass','scrambles','sacked'):
            pbp_data.passer_player_name.loc[i] = desc[j-1]
            break

In [ ]:
#Repeat process for receivers
#Make list of plays with missing receivers
missing_receivers = pbp_data.loc[(pbp_data.play_type=='no_play') & 
                                 (pbp_data.dropback==1) &
                                 (pbp_data.desc.str.contains(' pass')) &
                                 (pbp_data.penalty==0)].index

#Go through each play with missing receiver name
for i in missing_receivers:
    #Split string on spaces 
    desc = pbp_data.desc.loc[i].split()
    
    #Search each word of the description
    for j in range(0,len(desc)):
        #Attach suffix to abbreviated player name
        if desc[j] in ('Jr','Jr.','Sr','Sr.','II','III','IV'):
            desc[j] = desc[j-1] + ' ' + desc[j] 
    
    for k in range(0,len(desc)):
        #Receiver name comes after 'to'
        if desc[k] == 'to':
            pbp_data.receiver_player_name.loc[i] = desc[k+1]
            break
            
        elif desc[k] == 'intended':
            pbp_data.receiver_player_name.loc[i] = desc[k+2]
            break

In [ ]:
#Repeat process for rushers
#Make list of plays with missing rushers
missing_rushers = pbp_data.loc[(pbp_data.play_type=='no_play') & 
                               (pbp_data.rush==1)].index

#Go through each play with missing rusher name
for i in missing_rushers:
    #Split string on spaces 
    desc = pbp_data.desc.loc[i].split()
    
    #Search each word of the description
    for j in range(0,len(desc)):
        #Attach suffix to abbreviated player name
        if desc[j] in ('Jr','Jr.','Sr','Sr.','II','III','IV'):
            desc[j] = desc[j-1] + ' ' + desc[j] 
        #Passer name comes before pass, scramble, or sack
        if desc[j] in ('left','up','right'):
            pbp_data.rusher_player_name.loc[i] = desc[j-1]
            break

In [ ]:
#Remove incorrect periods at end of names
#Don't include names ending in Jr. or Sr.
pbp_data.receiver_player_name.loc[
    (pbp_data.receiver_player_name.str[-3:].isin(['Jr.','Sr.'])==False) &
    (pbp_data.receiver_player_name.str[-1]=='.')] = pbp_data.receiver_player_name.str[:-1]

In [ ]:
#Correct name discrepancy
pbp_data.passer_player_name.loc[pbp_data.passer_player_name=='G.Minshew II'] = 'G.Minshew'
pbp_data.rusher_player_name.loc[pbp_data.rusher_player_name=='G.Minshew II'] = 'G.Minshew'
pbp_data.receiver_player_name.loc[pbp_data.receiver_player_name=='G.Minshew II'] = 'G.Minshew'

pbp_data.passer_player_name.loc[pbp_data.passer_player_name=='D.Chark Jr.'] = 'D.Chark'
pbp_data.rusher_player_name.loc[pbp_data.rusher_player_name=='D.Chark Jr.'] = 'D.Chark'
pbp_data.receiver_player_name.loc[pbp_data.receiver_player_name=='D.Chark Jr.'] = 'D.Chark'

In [ ]:
#Optional change play_type to match dropback==1 or rush==1
#Retains original naming scheme of pass or run rather than dropback or rush
pbp_data.play_type.loc[pbp_data.dropback==1] = 'pass'
pbp_data.play_type.loc[pbp_data.rush==1] = 'run'

In [ ]:
#Change team abbreviations in older seasons
#STL to LA, SD to LAC, JAC to JAX
pbp_data.loc[(pbp_data.posteam=='STL')|(pbp_data.defteam=='STL')|
             (pbp_data.home_team=='STL')|(pbp_data.away_team=='STL')|
             (pbp_data.penalty_team=='STL')] = 'LA'

pbp_data.loc[(pbp_data.posteam=='SD')|(pbp_data.defteam=='SD')|
             (pbp_data.home_team=='SD')|(pbp_data.away_team=='SD')|
             (pbp_data.penalty_team=='SD')] = 'LAC'

pbp_data.loc[(pbp_data.posteam=='JAC')|(pbp_data.defteam=='JAC')|
             (pbp_data.home_team=='JAC')|(pbp_data.away_team=='JAC')|
             (pbp_data.penalty_team=='JAC')] = 'JAX'

In [ ]:
#Alphabetize columns
pbp_data.sort_index(axis=1,inplace=True)

In [ ]:
#Set path/file name to output cleaned CSV
pbp_data.to_csv('cleaned_pbp_data_' + str(YEAR) + '.csv',index=False)